In [1]:
import MySQLdb

In [2]:
db = MySQLdb.connect(
    host="public id",
    user="id",
    passwd="pw",
    db="world",
    charset="utf8")

db

<_mysql.connection open to '52.78.107.76' at 0000022ADE058EA8>

In [3]:
Query = """
    Show Tables;
"""

In [4]:
curs = db.cursor()
curs.execute(Query)

10

### 1) 면적이 10000km^2 이상인 국가의 인구밀도(1km^2 당 인구수)를 구하고 인구밀도가 200이상인국가들의 사용하고 있는 언어수가 5가지 이상인 나라를 조회 하세요.

In [5]:
Query = """
    SELECT NAME, COUNT(LANGUAGE) AS language_count
    FROM
    (
        SELECT code, name, ROUND(Population/SurfaceArea) AS density
        FROM country
        WHERE SurfaceArea > 10000
        having density > 200
        ORDER BY density  DESC) AS country
    JOIN countrylanguage
    ON countrylanguage.countrycode = country.code
    GROUP BY NAME
    HAVING language_count > 5
    ORDER BY language_count desc

"""

In [6]:
df = pd.read_sql(Query, db)
df

,NAME,language_count
0,India,12
1,Philippines,10
2,Vietnam,9
3,Bangladesh,7
4,Germany,6
5,Taiwan,6
6,Japan,6
7,Belgium,6


### 2) 사용하는 언어가 3가지 이하인 국가중 도시인구가 300만 이상인 도시를 아래와 같이 조회하세요.

In [11]:
Query = """
SELECT city.countrycode, city.name AS city_name, city.population,
country.name, city.language_count, languages
FROM
(
    SELECT city.countrycode, city.name, city.population, 
        cl.language_count, cl.languages
    FROM(
                SELECT countrycode, GROUP_CONCAT(LANGUAGE) AS languages,
                count(LANGUAGE) AS language_count
                FROM countrylanguage
                GROUP BY countrycode
                HAVING language_count <= 3) AS cl
        JOIN (
            SELECT *
            from city
            where population > 3000000
            ) as city

        ON city.countrycode = cl.countrycode
        ) AS city

JOIN country
ON country.code =  city.countrycode
ORDER BY population DESC

"""

In [12]:
df = pd.read_sql(Query, db)
df

,CountryCode,city_name,Population,name,language_count,languages
0,KOR,Seoul,9981619,South Korea,2,"Chinese,Korean"
1,TUR,Istanbul,8787958,Turkey,3,"Arabic,Kurdish,Turkish"
2,GBR,London,7285000,United Kingdom,3,"English,Gaeli,Kymri"
3,EGY,Cairo,6789479,Egypt,2,"Arabic,Sinaberberi"
4,PER,Lima,6464693,Peru,3,"Aimará,Ketšua,Spanish"
5,SGP,Singapore,4017733,Singapore,3,"Chinese,Malay,Tamil"
6,KOR,Pusan,3804522,South Korea,2,"Chinese,Korean"
7,EGY,Alexandria,3328196,Egypt,2,"Arabic,Sinaberberi"
8,SAU,Riyadh,3324000,Saudi Arabia,1,Arabic
9,TUR,Ankara,3038159,Turkey,3,"Arabic,Kurdish,Turkish"
